In [17]:
!pip install transformers==4.28.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification,Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import torch
import random
import re
from collections import Counter

In [19]:
dataset = pd.read_csv('Drug_Data.csv')

In [20]:
dataset

,drugName,Prescribed_for,Drug_Review,User_Rating,Date,Count_of_Reviews
0,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...",10,28-Feb-12,22
1,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done ...",8,17-May-09,17
2,Bactrim,Urinary Tract Infection,"""Quick reduction of symptoms""",9,29-Sep-17,3
3,Contrave,Weight Loss,"""Contrave combines drugs that were used for al...",9,05-Mar-17,35
4,Cyclafem 1 / 35,Birth Control,"""I have been on this birth control for one cyc...",9,22-Oct-15,4
...,...,...,...,...,...,...
53761,Tamoxifen,"Breast Cancer, Prevention","""I have taken Tamoxifen for 5 years. Side effe...",10,13-Sep-14,43
53762,Escitalopram,Anxiety,"""I&#039;ve been taking Lexapro (escitaploprgra...",9,08-Oct-16,11
53763,Levonorgestrel,Birth Control,"""I&#039;m married, 34 years old and I have no ...",8,15-Nov-10,7
53764,Tapentadol,Pain,"""I was prescribed Nucynta for severe neck/shou...",1,28-Nov-11,20


In [21]:
dataset = dataset.iloc[:10000,:]

In [22]:
dataset.columns

Index(['drugName', 'Prescribed_for', 'Drug_Review', 'User_Rating', 'Date',
       'Count_of_Reviews'],
      dtype='object')

In [23]:
dataset.isnull().sum()

drugName             0
Prescribed_for      48
Drug_Review          0
User_Rating          0
Date                 0
Count_of_Reviews     0
dtype: int64

In [24]:
dataset['Prescribed_for'] = dataset['Prescribed_for'].fillna("")

<ipython-input-24-2ee8da2cc52c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['Prescribed_for'] = dataset['Prescribed_for'].fillna("")


In [25]:
dataset.isnull().sum()

drugName            0
Prescribed_for      0
Drug_Review         0
User_Rating         0
Date                0
Count_of_Reviews    0
dtype: int64

In [26]:
dataset.drop('Date',axis='columns', inplace=True)

<ipython-input-26-6c585e0a5a17>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.drop('Date',axis='columns', inplace=True)


In [27]:
dataset.columns

Index(['drugName', 'Prescribed_for', 'Drug_Review', 'User_Rating',
       'Count_of_Reviews'],
      dtype='object')

In [28]:
dataset.duplicated().sum()

0

In [29]:
dataset['Prescribed_for'][1]

"Crohn's Disease, Maintenance"

In [30]:
dataset['Drug_Review'][1]

'"My son has Crohn&#039;s disease and has done very well on the Asacol.  He has no complaints and shows no side effects.  He has taken as many as nine tablets per day at one time.  I&#039;ve been very happy with the results, reducing his bouts of diarrhea drastically."'

In [31]:
dataset['drugName'][1]

'Mesalamine'

In [32]:
dataset["Prescribed_for"]

0                              Depression
1            Crohn's Disease, Maintenance
2                 Urinary Tract Infection
3                             Weight Loss
4                           Birth Control
                      ...                
9995                             Insomnia
9996    Upper Respiratory Tract Infection
9997              Emergency Contraception
9998                                 Pain
9999                        Endometriosis
Name: Prescribed_for, Length: 10000, dtype: object

In [33]:
categories = dataset["drugName"].values

In [34]:
my_list=[]
for i in categories:
  x = i.split(",")
  for j in x:
    my_list.append(j)


In [35]:
my_list

['Mirtazapine',
 'Mesalamine',
 'Bactrim',
 'Contrave',
 'Cyclafem 1 / 35',
 'Zyclara',
 'Copper',
 'Amitriptyline',
 'Methadone',
 'Levora',
 'Paroxetine',
 'Miconazole',
 'Belviq',
 'Seroquel',
 'Ambien',
 'Nuvigil',
 'Chantix',
 'Microgestin Fe 1 / 20',
 'Klonopin',
 'Ciprofloxacin',
 'Trazodone',
 'EnteraGam',
 'Aripiprazole',
 'Cyclosporine',
 'Oxybutynin',
 'Lurasidone',
 'Clonazepam',
 'Ciclopirox',
 'Sodium oxybate',
 'Lamotrigine',
 'Blisovi Fe 1 / 20',
 'Ivermectin',
 'Suprep Bowel Prep Kit',
 'Movantik',
 'Lorcaserin',
 'Actos',
 'Duloxetine',
 'NuvaRing',
 'Escitalopram',
 'Tesamorelin',
 'Campral',
 'Gabapentin',
 'Levonorgestrel',
 'Aubra',
 'Plan B One-Step',
 'Ethinyl estradiol / etonogestrel',
 'Microgestin Fe 1.5 / 30',
 'Wellbutrin',
 'Clonazepam',
 'Benzoyl peroxide / clindamycin',
 'Etonogestrel',
 'Miconazole',
 'Nitrofurantoin',
 'Etonogestrel',
 'Ortho Tri-Cyclen Lo',
 'Tamsulosin',
 'Tofacitinib',
 'Cryselle',
 'Amphetamine / dextroamphetamine',
 'Clindamycin',

In [36]:
pattern = r"\d+</span>.*"

# Remove elements matching the pattern
filtered_list = [item for item in my_list if not re.match(pattern, item)]

print(filtered_list)

['Mirtazapine', 'Mesalamine', 'Bactrim', 'Contrave', 'Cyclafem 1 / 35', 'Zyclara', 'Copper', 'Amitriptyline', 'Methadone', 'Levora', 'Paroxetine', 'Miconazole', 'Belviq', 'Seroquel', 'Ambien', 'Nuvigil', 'Chantix', 'Microgestin Fe 1 / 20', 'Klonopin', 'Ciprofloxacin', 'Trazodone', 'EnteraGam', 'Aripiprazole', 'Cyclosporine', 'Oxybutynin', 'Lurasidone', 'Clonazepam', 'Ciclopirox', 'Sodium oxybate', 'Lamotrigine', 'Blisovi Fe 1 / 20', 'Ivermectin', 'Suprep Bowel Prep Kit', 'Movantik', 'Lorcaserin', 'Actos', 'Duloxetine', 'NuvaRing', 'Escitalopram', 'Tesamorelin', 'Campral', 'Gabapentin', 'Levonorgestrel', 'Aubra', 'Plan B One-Step', 'Ethinyl estradiol / etonogestrel', 'Microgestin Fe 1.5 / 30', 'Wellbutrin', 'Clonazepam', 'Benzoyl peroxide / clindamycin', 'Etonogestrel', 'Miconazole', 'Nitrofurantoin', 'Etonogestrel', 'Ortho Tri-Cyclen Lo', 'Tamsulosin', 'Tofacitinib', 'Cryselle', 'Amphetamine / dextroamphetamine', 'Clindamycin', 'Pramipexole', 'Skyla', 'Lastacaft', 'Effexor XR', 'Nifedi

In [37]:
word_counts = Counter(filtered_list)
for word, count in word_counts.items():
  print(f"{word}: {count}")

Mirtazapine: 36
Mesalamine: 9
Bactrim: 19
Contrave: 53
Cyclafem 1 / 35: 3
Zyclara: 3
Copper: 33
Amitriptyline: 47
Methadone: 19
Levora: 8
Paroxetine: 42
Miconazole: 64
Belviq: 31
Seroquel: 17
Ambien: 26
Nuvigil: 14
Chantix: 39
Microgestin Fe 1 / 20: 22
Klonopin: 31
Ciprofloxacin: 23
Trazodone: 35
EnteraGam: 1
Aripiprazole: 32
Cyclosporine: 3
Oxybutynin: 6
Lurasidone: 22
Clonazepam: 55
Ciclopirox: 2
Sodium oxybate: 3
Lamotrigine: 30
Blisovi Fe 1 / 20: 3
Ivermectin: 5
Suprep Bowel Prep Kit: 56
Movantik: 2
Lorcaserin: 37
Actos: 3
Duloxetine: 68
NuvaRing: 31
Escitalopram: 79
Tesamorelin: 1
Campral: 8
Gabapentin: 75
Levonorgestrel: 241
Aubra: 10
Plan B One-Step: 25
Ethinyl estradiol / etonogestrel: 34
Microgestin Fe 1.5 / 30: 6
Wellbutrin: 22
Benzoyl peroxide / clindamycin: 16
Etonogestrel: 215
Nitrofurantoin: 24
Ortho Tri-Cyclen Lo: 12
Tamsulosin: 10
Tofacitinib: 2
Cryselle: 8
Amphetamine / dextroamphetamine: 24
Clindamycin: 15
Pramipexole: 9
Skyla: 52
Lastacaft: 2
Effexor XR: 29
Nifedipin

In [38]:
categories = filtered_list #dataset["Prescribed_for"].values
id2label = {idx:label for idx, label in enumerate(filtered_list)}
label2id = {label:idx for idx, label in enumerate(filtered_list)}
filtered_list

['Mirtazapine',
 'Mesalamine',
 'Bactrim',
 'Contrave',
 'Cyclafem 1 / 35',
 'Zyclara',
 'Copper',
 'Amitriptyline',
 'Methadone',
 'Levora',
 'Paroxetine',
 'Miconazole',
 'Belviq',
 'Seroquel',
 'Ambien',
 'Nuvigil',
 'Chantix',
 'Microgestin Fe 1 / 20',
 'Klonopin',
 'Ciprofloxacin',
 'Trazodone',
 'EnteraGam',
 'Aripiprazole',
 'Cyclosporine',
 'Oxybutynin',
 'Lurasidone',
 'Clonazepam',
 'Ciclopirox',
 'Sodium oxybate',
 'Lamotrigine',
 'Blisovi Fe 1 / 20',
 'Ivermectin',
 'Suprep Bowel Prep Kit',
 'Movantik',
 'Lorcaserin',
 'Actos',
 'Duloxetine',
 'NuvaRing',
 'Escitalopram',
 'Tesamorelin',
 'Campral',
 'Gabapentin',
 'Levonorgestrel',
 'Aubra',
 'Plan B One-Step',
 'Ethinyl estradiol / etonogestrel',
 'Microgestin Fe 1.5 / 30',
 'Wellbutrin',
 'Clonazepam',
 'Benzoyl peroxide / clindamycin',
 'Etonogestrel',
 'Miconazole',
 'Nitrofurantoin',
 'Etonogestrel',
 'Ortho Tri-Cyclen Lo',
 'Tamsulosin',
 'Tofacitinib',
 'Cryselle',
 'Amphetamine / dextroamphetamine',
 'Clindamycin',

In [39]:
model_name = "dmis-lab/biobert-base-cased-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(filtered_list),  problem_type="multi_label_classification",id2label=id2label,label2id=label2id )

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.1 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification we

In [40]:
def map_Prescribed(drug_prescribed):
  drug_prescribed = [pres for pres in drug_prescribed if pres in filtered_list]
  if len(drug_prescribed) == 0:
    drug_prescribed1 = []

  label = [0]*len(filtered_list)

  for pres in drug_prescribed:
    index = filtered_list.index(pres)
    label[index] = 1
  return label

In [41]:
# filtered_list

In [42]:
dataset['drugName']

0           Mirtazapine
1            Mesalamine
2               Bactrim
3              Contrave
4       Cyclafem 1 / 35
             ...       
9995             Ativan
9996       Erythromycin
9997             Plan B
9998           Dilaudid
9999         Leuprolide
Name: drugName, Length: 10000, dtype: object

In [43]:
map_Prescribed(['Mirtazapine', 'Tapentadol'])

[1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [44]:
def clean_text(text):
    # Remove digits concatenated with words
    text = re.sub(r'\b\d+\w+', '', text)

    # Remove non-alphanumeric characters and extra whitespaces
    text = re.sub(r'\W+', ' ', text.strip())

    # Convert to lowercase
    text = text.lower()

    return text

In [45]:
clean_text("I&#039;ve had the copper coil for about 3 months now. I was really excited at the thought of not taking hormones. I&#039;m good with pain however I nearly fainted with insertion, couldn&#039;t belive how painful it was; the doctor did say it is very painful for some. Well 3 months in, my periods last 11 days and I&#039;m in pain for about 15 days with random twangs especially in the left side and I&#039;m considering whether I want to put up with the intense pain and heavy periods. I&#039;d recommend this 100% to somebody who doesn&#039;t already have heavy painful periods but right now it just isn&#039;t for me")

'i ve had the copper coil for about 3 months now i was really excited at the thought of not taking hormones i m good with pain however i nearly fainted with insertion couldn t belive how painful it was the doctor did say it is very painful for some well 3 months in my periods last days and i m in pain for about days with random twangs especially in the left side and i m considering whether i want to put up with the intense pain and heavy periods i d recommend this to somebody who doesn t already have heavy painful periods but right now it just isn t for me'

In [46]:
# dataset

In [47]:
# len(dataset['drugName'].unique())

In [48]:
dataset.columns

Index(['drugName', 'Prescribed_for', 'Drug_Review', 'User_Rating',
       'Count_of_Reviews'],
      dtype='object')

In [49]:
# Define the dataset class
class PrescribedDataset(Dataset):
  def __init__(self, dataset, tokenizer):
    self.Prescribed = dataset['Prescribed_for']
    self.targets = list(map(map_Prescribed,dataset['drugName']))
    self.tokenizer = tokenizer
  
  def __len__(self):
    return len(self.Prescribed)
  
  def __getitem__(self, index):
    Prescribed = self.Prescribed[index]
    target = self.targets[index]
    
    cleaned = clean_text(Prescribed)
    inputs = self.tokenizer(cleaned, padding='max_length', truncation=True, max_length=128, return_tensors='pt')
    label = torch.tensor(target)
    
    return {
            'input_ids': inputs['input_ids'][0],
            'attention_mask': inputs['attention_mask'][0],
            'labels': label.float()
        }

In [50]:
!nvidia-smi

Sun May 21 13:50:05 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [51]:
train_dataset = PrescribedDataset(dataset, tokenizer)

In [52]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=4,
    learning_rate=2e-5,
    save_steps=1000
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

In [53]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [54]:
# Load the test dataset
test = pd.read_csv("/content/Drug_Data.csv")
test.head()

,drugName,Prescribed_for,Drug_Review,User_Rating,Date,Count_of_Reviews
0,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...",10,28-Feb-12,22
1,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done ...",8,17-May-09,17
2,Bactrim,Urinary Tract Infection,"""Quick reduction of symptoms""",9,29-Sep-17,3
3,Contrave,Weight Loss,"""Contrave combines drugs that were used for al...",9,05-Mar-17,35
4,Cyclafem 1 / 35,Birth Control,"""I have been on this birth control for one cyc...",9,22-Oct-15,4


In [55]:
test = test.iloc[:10000,:]

In [56]:
test['Prescribed_for'].isna().sum()

48

In [57]:
test.dropna(inplace=True)

In [58]:
test['Prescribed_for'].isna().sum()

0

In [59]:
test['Prescribed_for'] = test['Prescribed_for'].apply(clean_text)

In [72]:
test['Prescribed_for']

0                              depression
1             crohn s disease maintenance
2                 urinary tract infection
3                             weight loss
4                           birth control
                      ...                
9995                             insomnia
9996    upper respiratory tract infection
9997              emergency contraception
9998                                 pain
9999                        endometriosis
Name: Prescribed_for, Length: 9952, dtype: object

In [73]:
def preprocess_Prescribed(Prescribed):
  inputs = tokenizer(Prescribed, padding='max_length', truncation=True, max_length=128, return_tensors='pt')
  return {
      'input_ids': inputs['input_ids'][0],
      'attention_mask': inputs['attention_mask'][0]
      }

In [76]:
test_inputs = [preprocess_Prescribed(Prescribed) for Prescribed in test['Prescribed_for'].tolist()]
test_outputs = trainer.predict(test_inputs)

In [ ]:
y_pred

In [63]:
test_outputs.predictions[:5]

array([[-0.47814775, -0.22736786, -0.08655672, ...,  0.43750483,
         0.13414145,  0.08790956],
       [-0.01264817, -0.6072767 , -0.02565673, ..., -0.18255748,
         0.54354966, -0.16277109],
       [-0.19629303, -0.5293759 , -0.06843677, ...,  0.03132373,
         0.29797363, -0.07294921],
       [-0.04581891, -0.5828301 , -0.00249807, ..., -0.07911257,
         0.43003237, -0.18877389],
       [-0.0083256 , -0.5985617 ,  0.0197478 , ..., -0.10689578,
         0.45487326, -0.21978165]], dtype=float32)

In [87]:
def convert_predictions(preds):

  sigmoid = torch.nn.Sigmoid()
  probs = sigmoid(torch.Tensor(preds))
  y_pred = np.zeros(probs.shape)
  y_pred[np.where(probs >= 0.50)] = 1
  
  return y_pred

In [88]:
y_pred = convert_predictions(test_outputs.predictions)

In [89]:
y_pred[:5]

array([[0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 1., 0.]])

In [98]:
Prescribed = 'urinary tract infection'
cleaned_Prescribed = clean_text(Prescribed)
Prescribed_inputs = preprocess_Prescribed(cleaned_Prescribed)
Prescribed_outputs = trainer.predict([Prescribed_inputs])
Prescribed_pred  = convert_predictions(Prescribed_outputs.predictions)
Prescribed_pred

array([[0., 0., 0., ..., 1., 1., 0.]])

In [99]:
# test

In [100]:
def Recommend_similar_quotes(inputs_text,Prescribed,list_Prescribed):

  similarity_scores = cosine_similarity(list_Prescribed, Prescribed)

  # Sort the rows in descending order based on the similarity score
  indices = np.argsort(similarity_scores, axis=0)[::-1].squeeze()

  # Print the rows in descending order of similarity
  for i in indices[:10]:
    print(' Prescribed:',inputs_text[i],'\n','Score:',similarity_scores[i])
    print("\n")
    print("*"*100)

In [101]:
Recommend_similar_quotes(test['drugName'].tolist(),Prescribed_pred,y_pred)

 Prescribed: Macrobid 
 Score: [1.]


****************************************************************************************************
 Prescribed: Sulfamethoxazole / trimethoprim 
 Score: [1.]


****************************************************************************************************
 Prescribed: Nitrofurantoin 
 Score: [1.]


****************************************************************************************************
 Prescribed: Hiprex 
 Score: [1.]


****************************************************************************************************
 Prescribed: Sulfamethoxazole / trimethoprim 
 Score: [1.]


****************************************************************************************************
 Prescribed: Sulfamethoxazole / trimethoprim 
 Score: [1.]


****************************************************************************************************
 Prescribed: Sulfamethoxazole / trimethoprim 
 Score: [1.]


********************************